In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import os
import glob
import itertools
import matplotlib.pyplot as plt

In [2]:
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}

In [3]:
list_halaman = [*range(1,101)]
list_tempat = ['dki-jakarta','bekasi','tangerang','bogor','depok']
list_kategori = ['apartemen', 'rumah']
list_tempat_baru = ['jakarta-barat/cengkareng','jakarta-barat/kalideres','jakarta-barat/kembangan','jakarta-barat/palmerah','jakarta-barat/tamansari','jakarta-barat/tambora','jakarta-pusat/cempaka putih','jakarta-pusat/gambir','jakarta-pusat/johar baru','jakarta-pusat/kemayoran','jakarta-pusat/sawah besar','jakarta-pusat/senen','jakarta-selatan/cilandak','jakarta-selatan/jagakarsa','jakarta-selatan/kebayoran baru','jakarta-selatan/mampang prapatan','jakarta-selatan/pancoran','jakarta-selatan/pasar minggu','jakarta-selatan/pesanggrahan','jakarta-selatan/setia budi','jakarta-selatan/tebet','jakarta-timur/cakung','jakarta-timur/cipayung','jakarta-timur/ciracas','jakarta-timur/duren sawit','jakarta-timur/jatinegara','jakarta-timur/kramatjati','jakarta-timur/makasar','jakarta-timur/matraman','jakarta-utara/cilincing','jakarta-utara/kelapa gading','jakarta-utara/koja','jakarta-utara/pademangan','jakarta-utara/penjaringan','jakarta-utara/tanjung priok','jakarta-barat/grogolpetamburan','jakarta-barat/kebonjeruk','jakarta-pusat/menteng','jakarta-pusat/tanahabang','jakarta-selatan/setiabudi','jakarta-timur/pasarrebo','jakarta-timur/pulogadung','jakarta-selatan/kebayoran lama']

In [4]:
path = r'C:\Users\PutuAndika\OneDrive - Migo\Desktop\Web Scraping\Rumah'
path_sewa = r'C:\Users\PutuAndika\OneDrive - Migo\Desktop\Web Scraping\Rumah\Sewa'

In [5]:
for halaman, tempat, kategori in itertools.product(list_halaman,list_tempat, list_kategori):
    url = f'https://www.rumah123.com/jual/{str(tempat)}/{kategori}/?page={str(halaman)}#qid~42eeef39-13df-4144-afa7-f1229e9827be'
    r = requests.get(url, headers = header)
    c = r.content
    page = BeautifulSoup(c, 'html.parser')
    all_tempat = page.find_all('div',class_='ui-organism-intersection__element intersection-card-container')
    place_name = []
    address = []
    lb = []
    kt = []
    price = []
    mortage = []
    date_posted = []
    tayang = []
    for i in all_tempat:
        try:
            name = i.find('a').find('h2').text
        except:
            name = None
        try:
            alamat = i.find('p', class_= 'card-featured__middle-section__location').text
        except:
            alamat = None
        try:
            kamar = i.find('span', class_ = 'attribute-text').text
        except:
            kamar = None
        try:
            luas = i.find('div', class_ = 'attribute-info').find('span').text
        except:
            luas = None
        try:
            tes_harga = i.find('div', class_='card-featured__middle-section__price').find('strong').text
        except:
            tes_harga = None
        try:
            cicilan = i.find('div', class_='ui-organisms-card-r123-featured__middle-section__price').find('em').text.replace("Cicilan :","").strip()
        except:
            cicilan = None
        try:
            jadwal = i.find('div', class_='ui-organisms-card-r123-basic__bottom-section__agent').text
        except:
            jadwal = None
        place_name.append(name)
        address.append(alamat)
        kt.append(kamar)
        lb.append(luas)
        price.append(tes_harga)
        mortage.append(cicilan)
        tayang.append(jadwal)
        df = pd.DataFrame({'Place Name':place_name, 'Address':address, 'Bedroom':kt, 'Area':lb,
                            'Price':price, 'Mortage':mortage, 'Category': kategori, 'Tayang':tayang})
        df.to_csv(os.path.join(path, f'{halaman}_{tempat}_{kategori}.csv'), index=False, sep=';')

KeyboardInterrupt: 

In [4]:
path = r'C:\Users\PutuAndika\OneDrive - Migo\Desktop\Web Scraping\Rumah'
path_output = r'C:\Users\PutuAndika\OneDrive - Migo\Desktop\Web Scraping\Rumah\Combined'
os.chdir(path)
out = glob.glob('*.csv')
df2 = pd.concat([pd.read_csv(f,sep = ';') for f in out])

In [5]:
del df2['Mortage']

In [6]:
df2 = df2.dropna()

In [7]:
df2.columns = [i.replace(" ","_") for i in df2.columns]

In [8]:
def angka(a):
    if "Miliar" in a:
        return 1000
    elif "Triliun" in a:
        return 1000000
    else:
        return 1

In [9]:
df2['Angka'] = df2.Price.apply(lambda x: angka(x))

In [10]:
df2.Price = df2.Price.str.replace(",",".")

In [11]:
df2.Price = df2.Price.str.replace("Miliar","").str.replace("Rp","").str.replace("Juta","").str.replace("Triliun","").str.strip()

In [12]:
df2.Price.unique()

array(['375', '570', '1.4', ..., '578', '246', '2.26'], dtype=object)

In [14]:
df2.Price = df2.Price.astype('float')
df2.head()

,Place_Name,Address,Bedroom,Area,Price,Category,Tayang,Angka
0,Rumah 1 Lantai Minimalis Modern di Polu timaha Baru Selatan HGB,"Babelan, Bekasi",2.0,60 m²,375.0,rumah,"Yusuf Hidayatullah Tayang Sejak 15 November, 2022",1
1,Dijual Rumah Siap Huni di Bumi Bekasi Baru Rawalumbu,"Rawalumbu, Bekasi",2.0,84 m²,570.0,rumah,"Elly Sefa Widyarini Tayang Sejak 15 November, 2022",1
2,Rumah di Prima Harapan Regency Bekasi,"Prima Harapan, Bekasi",3.0,133 m²,1.4,rumah,"Henny Era Sky Summarecon BekasiTayang Sejak 15 November, 2022",1000
3,Rumah Baru Murah Pekayon Galaxy Bekasi,"Pekayon, Bekasi",3.0,150 m²,1.6,rumah,"Gus SusiantoTayang Sejak 15 November, 2022",1000
4,Dijual Cepat Rumah Siap Huni Full Renov Cluster Ifolia Harapan Indah,"Harapan Indah, Bekasi",3.0,144 m²,1.4,rumah,"Evi Rosalina Tayang Sejak 15 November, 2022",1000


In [15]:
df2['Price (in Million)'] = df2.Price * df2.Angka

In [16]:
df2.head()

,Place_Name,Address,Bedroom,Area,Price,Category,Tayang,Angka,Price (in Million)
0,Rumah 1 Lantai Minimalis Modern di Polu timaha Baru Selatan HGB,"Babelan, Bekasi",2.0,60 m²,375.0,rumah,"Yusuf Hidayatullah Tayang Sejak 15 November, 2022",1,375.0
1,Dijual Rumah Siap Huni di Bumi Bekasi Baru Rawalumbu,"Rawalumbu, Bekasi",2.0,84 m²,570.0,rumah,"Elly Sefa Widyarini Tayang Sejak 15 November, 2022",1,570.0
2,Rumah di Prima Harapan Regency Bekasi,"Prima Harapan, Bekasi",3.0,133 m²,1.4,rumah,"Henny Era Sky Summarecon BekasiTayang Sejak 15 November, 2022",1000,1400.0
3,Rumah Baru Murah Pekayon Galaxy Bekasi,"Pekayon, Bekasi",3.0,150 m²,1.6,rumah,"Gus SusiantoTayang Sejak 15 November, 2022",1000,1600.0
4,Dijual Cepat Rumah Siap Huni Full Renov Cluster Ifolia Harapan Indah,"Harapan Indah, Bekasi",3.0,144 m²,1.4,rumah,"Evi Rosalina Tayang Sejak 15 November, 2022",1000,1400.0


In [17]:
del df2['Angka']

In [18]:
df2[['Kecamatan','Kota']] = df2.Address.str.strip().str.split(",",expand=True)

In [19]:
del df2['Address']

In [20]:
df2[['Area (m2)','Satuan']] = df2['Area'].str.split("m",expand=True)

In [21]:
df2['Area (m2)'] = df2['Area (m2)'].astype('float')

In [22]:
def angka_cicilan(a):
    if "Jutaan per bulan" in a:
        return 1
    elif "Miliaran per bulan" in a:
        return 1000
    else:
        return 0.001

In [23]:
df2['Price per Area'] = df2['Price (in Million)'] / df2['Area (m2)']

In [24]:
df2[['Nama','Date Posted']] = df2['Tayang'].str.split("Tayang Sejak", expand=True)

In [25]:
df2 = df2.drop(columns = ['Tayang', 'Satuan','Nama'])

In [26]:
df2['Date Posted'] = df2['Date Posted'].str.strip().str.replace(",","").str.strip()

In [27]:
df2[['Day','Month','Year']] = df2['Date Posted'].str.split(" ", expand=True)

In [28]:
list_month = ['11', '10', '12', '09', '08', '07',
       '06', '05', '04', '03', '02', '01', None]

In [29]:
df2['Month'] = df2['Month'].replace(df2['Month'].unique(), list_month)

In [30]:
df2['Date'] = df2['Day'] + '/' +df2['Month']+'/'+df2['Year']

In [31]:
df2.head()

,Place_Name,Bedroom,Area,Price,Category,Price (in Million),Kecamatan,Kota,Area (m2),Price per Area,Date Posted,Day,Month,Year,Date
0,Rumah 1 Lantai Minimalis Modern di Polu timaha Baru Selatan HGB,2.0,60 m²,375.0,rumah,375.0,Babelan,Bekasi,60.0,6.250000,15 November 2022,15,11,2022,15/11/2022
1,Dijual Rumah Siap Huni di Bumi Bekasi Baru Rawalumbu,2.0,84 m²,570.0,rumah,570.0,Rawalumbu,Bekasi,84.0,6.785714,15 November 2022,15,11,2022,15/11/2022
2,Rumah di Prima Harapan Regency Bekasi,3.0,133 m²,1.4,rumah,1400.0,Prima Harapan,Bekasi,133.0,10.526316,15 November 2022,15,11,2022,15/11/2022
3,Rumah Baru Murah Pekayon Galaxy Bekasi,3.0,150 m²,1.6,rumah,1600.0,Pekayon,Bekasi,150.0,10.666667,15 November 2022,15,11,2022,15/11/2022
4,Dijual Cepat Rumah Siap Huni Full Renov Cluster Ifolia Harapan Indah,3.0,144 m²,1.4,rumah,1400.0,Harapan Indah,Bekasi,144.0,9.722222,15 November 2022,15,11,2022,15/11/2022


In [32]:
df2['Date'] = pd.to_datetime(df2['Date'])

c:\Users\PutuAndika\anaconda3\envs\geo_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\PutuAndika\anaconda3\envs\geo_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '25/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\PutuAndika\anaconda3\envs\geo_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\PutuAndika\anaconda3\envs\geo_env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarn

In [43]:
df2.to_excel(os.path.join(path_output,'Cleaned V2.xlsx'), index = False) 